In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader
import dataset as d
import model as m
import trainer as t
import util 
import eval
import loss as l
import config 
from importlib import reload


reload(config)
reload(m)
reload(d)
reload(l)
reload(util)
reload(t)
reload(eval)

de = util.DataExtractor()
gt_df = de.croppedData()


dataset = d.CroppedDataset(gt_df, config.img_dir)
dataset[12]

Saving to: c:\Users\alexh\Desktop\cv2\obb_anns_hausarbeit\ds2_dense\ds2_dense\gt_space.json


(tensor([[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]),
 tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          ...,
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
      

In [18]:
loader = DataLoader(dataset, batch_size=4, num_workers=1)

for imgs, tgts in loader:
    imgs = imgs.unsqueeze(1)
    print(imgs.shape)
    break

torch.Size([4, 1, 480, 480])


In [27]:
model = m.YOLOResNet()
name = "debug_training"
model = util.loadModel(name, model) #0.0122, 0.0059   #concorde0->concorde00

loader = DataLoader(dataset, batch_size=4, num_workers=1)
loss_fn = l.YOLOv2Loss(l_noobj = 0.10, l_wh = 1.0, l_obj = 4.0)
trainer = t.Trainer(model, loss_fn, gt_df, loader, epochs=10,  lr=1e-4, modelname=name, save_interval=1, device='cuda')

try:
    trainer.run()
except KeyboardInterrupt:
    print("Training interrupted. Saving model...")

util.saveModel(name, model)


c:\Users\alexh\Desktop\cv2\obb_anns_hausarbeit\util.py:234: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path, map_location=device))


[loadMode] Loaded weights from model_dumps\debug_training.pth
2000 crops processed


AttributeError: 'builtin_function_or_method' object has no attribute 'average_precision'